In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import numpy as np
from sklearn import metrics

In [2]:
df = pd.read_csv("battingstats.csv",low_memory=False,parse_dates=True)
df

,Player,Span,Mat,Inns,NO,Runs,HS,HS-N/O?,Ave,Hundreds,Fifties,Ducks,Batting Style,Player Role
0,SR Tendulkar (INDIA),1989-2013,200,329,33,15921,248,Yes,53.78,51,68,14,R,top-order batsman
1,RT Ponting (AUS),1995-2012,168,287,29,13378,257,No,51.85,41,62,17,R,top-order batsman
2,JH Kallis (ICC/SA),1995-2013,166,280,40,13289,224,No,55.37,45,58,16,R,allrounder
3,R Dravid (ICC/INDIA),1996-2012,164,286,32,13288,270,No,52.31,36,63,8,R,top-order batsman
4,AN Cook (ENG),2006-2018,161,291,16,12472,294,No,45.35,33,57,9,L,opening batsman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,SMH Kirmani (INDIA),1986-1986,1,-,-,-,-,No,-,-,-,-,R,NaN
1423,Shahid Mahboob (PAK),1989-1989,1,-,-,-,-,No,-,-,-,-,R,NaN
1424,L Sivaramakrishnan (INDIA),1986-1986,1,-,-,-,-,No,-,-,-,-,R,NaN
1425,Usman Shinwari (PAK),2019-2019,1,-,-,-,-,No,-,-,-,-,R,bowler


In [3]:
# First deal with Player and Span columns to extract nationality & Start and End dates
nationality = pd.Series([], dtype="int16")
player = pd.Series([], dtype="int16")
startdate = pd.Series([], dtype="int16")
enddate = pd.Series([], dtype="int16")

for index, name in enumerate(df.Player):
    # Find all occurrences of brackets. 
    startBrac = str(name).find("(")
    endBrac = str(name).find(")")
    nationality[index] = str(name)[startBrac+1:endBrac]
    player[index] = str(name)[:startBrac]
    while True:
        startBrac = str(name).find("(", endBrac)
        endBrac = str(name).find(")", startBrac)
        if startBrac != -1:
            if len(str(name)[startBrac+1:endBrac])>1:
                nationality[index] = str(name)[startBrac+1:endBrac]
        if startBrac == -1:
            break

for idx, date in enumerate(df.Span):
    dashloc = str(date).find("-")
    startdate[idx] = int(str(date)[:dashloc])
    enddate[idx] = int(str(date)[dashloc+1:])

df.drop(columns=["Player","Span"], inplace=True)
df["Nationality"] = nationality
df["Player"] = player
df["StartDate"] = startdate
df["EndDate"] = enddate

In [4]:
df.StartDate = df.StartDate.astype("int16")
df.EndDate = df.EndDate.astype("int16")

In [5]:
df

,Mat,Inns,NO,Runs,HS,HS-N/O?,Ave,Hundreds,Fifties,Ducks,Batting Style,Player Role,Nationality,Player,StartDate,EndDate
0,200,329,33,15921,248,Yes,53.78,51,68,14,R,top-order batsman,INDIA,SR Tendulkar,1989,2013
1,168,287,29,13378,257,No,51.85,41,62,17,R,top-order batsman,AUS,RT Ponting,1995,2012
2,166,280,40,13289,224,No,55.37,45,58,16,R,allrounder,ICC/SA,JH Kallis,1995,2013
3,164,286,32,13288,270,No,52.31,36,63,8,R,top-order batsman,ICC/INDIA,R Dravid,1996,2012
4,161,291,16,12472,294,No,45.35,33,57,9,L,opening batsman,ENG,AN Cook,2006,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,1,-,-,-,-,No,-,-,-,-,R,NaN,INDIA,SMH Kirmani,1986,1986
1423,1,-,-,-,-,No,-,-,-,-,R,NaN,PAK,Shahid Mahboob,1989,1989
1424,1,-,-,-,-,No,-,-,-,-,R,NaN,INDIA,L Sivaramakrishnan,1986,1986
1425,1,-,-,-,-,No,-,-,-,-,R,bowler,PAK,Usman Shinwari,2019,2019


In [6]:
# Analyse PlayerRole
df["Player Role"].unique()

array(['top-order batsman', 'allrounder', 'opening batsman',
       'wicketkeeper batsman', 'batsman', 'middle-order batsman', nan,
       'bowler', 'batting allrounder', 'bowling allrounder',
       'wicketkeeper'], dtype=object)

In [7]:
# Should I combine wicketkeeper & wicketkeeper batsman?
df2 = df.copy()
df2 = df2[df2.Ave != "-"]
df2.Ave = df2.Ave.astype("float16")

In [8]:
rolesummary = df2[["Ave", "Player Role"]].groupby("Player Role").mean()
rolesummary["St. Deviation"] = df2[["Ave", "Player Role"]].groupby("Player Role").std().Ave.values
rolesummary

,Ave,St. Deviation
Player Role,,
allrounder,25.109375,12.933654
batsman,31.609375,12.866581
batting allrounder,31.890625,14.810161
bowler,12.125000,8.609440
bowling allrounder,23.421875,12.662920
middle-order batsman,33.312500,19.507223
opening batsman,30.671875,12.227131
top-order batsman,33.781250,13.103186
wicketkeeper,21.421875,3.137527


In [9]:
# In the context of cricket, averages of 21 and 28 are poor, but are also significantly different from each other. 
# Given that the batting average for a wicketkeeper seems to vary quite a bit less than that of a wicketkeeper batsman, I choose to keep these classes separate
# There does appear to be a case to merge opening & top-order batsmen together -> don't merge
# As well as "allrounder" and "bowling allrounder" -> merge these
# Decisions made based on prior cricketing knowledge. Will make a model where I merge both, however.
playerRole = pd.Series([], dtype="object")
for index, role in enumerate(df["Player Role"]):
    if str(role) == "allrounder":
        playerRole[index] = "bowling allrounder"
    else:
        playerRole[index] = str(role)
        
df["Player Role"] = playerRole

In [10]:
df.Nationality.unique()

array(['INDIA', 'AUS', 'ICC/SA', 'ICC/INDIA', 'ENG', 'SL', 'ICC/WI', 'WI',
       'PAK', 'SA', 'ICC/PAK', 'NZ', 'ZIM', 'ICC/NZ', 'BDESH', 'ENG/ICC',
       'ICC/SL', 'AFG', 'IRE', 'ENG/IRE'], dtype=object)

In [11]:
# ICC teams aren't too important, so in theory I should merge ICC/SA with SA etc etc but let's see if there is any statistical base to this
df2 = df.copy()
df2 = df2[df2.Ave != "-"]
df2.Ave = df2.Ave.astype("float16")
rolesummary = df2[["Ave", "Nationality"]].groupby("Nationality").mean()
rolesummary["St. Deviation"] = df2[["Ave", "Nationality"]].groupby("Nationality").std().Ave.values
rolesummary["Count"] = df2[["Ave", "Nationality"]].groupby("Nationality").count().Ave.values
rolesummary

,Ave,St. Deviation,Count
Nationality,,,
AFG,17.859375,12.435806,18
AUS,25.296875,18.645583,137
BDESH,17.031250,12.232083,96
ENG,20.359375,11.961786,199
ENG/ICC,21.781250,14.125563,2
ENG/IRE,8.601562,NaN,1
ICC/INDIA,50.812500,2.099223,2
ICC/NZ,30.000000,NaN,1
ICC/PAK,49.593750,NaN,1


In [12]:
# First look at the NaNs. They are caused because we have only a single piece of data in each of those categories.
# Hence merge ICC/WI and WI, ICC/SL and SL, ICC/Pak and Pak, ICC/NZ and NZ and ENG/IRE with IRE
# Due to the variability in ENG/ICC and ICC/SA, merge these players into ENG and SA teas respectively
iccdict = {"ICC/WI":"WI","ICC/SL":"SL","ICC/PAK":"PAK","ICC/NZ":"NZ","ENG/IRE":"IRE","ENG/ICC":"ENG","ICC/SA":"SA"}
for key, value in iccdict.items():
    for index, nation in enumerate(df.Nationality):
        if nation == key:
            df.Nationality[index] = value

<ipython-input-12-661583b7ad36>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Nationality[index] = value


In [13]:
df.Nationality.unique()

array(['INDIA', 'AUS', 'SA', 'ICC/INDIA', 'ENG', 'SL', 'WI', 'PAK', 'NZ',
       'ZIM', 'BDESH', 'AFG', 'IRE'], dtype=object)

In [14]:
df["Batting Style"].unique() # Good

array(['R', 'L'], dtype=object)

In [15]:
# Now need to analyse the columns with an Average of "-"
df3 = df[df.Ave=="-"]
df3

,Mat,Inns,NO,Runs,HS,HS-N/O?,Ave,Hundreds,Fifties,Ducks,Batting Style,Player Role,Nationality,Player,StartDate,EndDate
1028,1,1,1,54,54,Yes,-,0,1,0,R,top-order batsman,AUS,SG Law,1995,1995
1236,1,2,2,16,12,Yes,-,0,0,0,R,bowler,PAK,Muhammad Musa,2019,2019
1262,3,4,4,13,8,Yes,-,0,0,0,R,nan,INDIA,T Yohannan,2001,2002
1296,1,1,1,9,9,Yes,-,0,0,0,R,nan,PAK,Iftikhar Anjum,2006,2006
1303,2,2,2,9,9,Yes,-,0,0,0,R,nan,PAK,Zakir Khan,1986,1989
1305,1,1,1,8,8,Yes,-,0,0,0,L,nan,AUS,PA Emery,1994,1994
1322,1,1,1,7,7,Yes,-,0,0,0,R,bowler,WI,DM Washington,2005,2005
1333,2,2,2,6,6,Yes,-,0,0,0,R,nan,AUS,SA Muller,1999,1999
1338,2,2,2,5,4,Yes,-,0,0,0,R,nan,WI,IBA Allen,1991,1991
1340,1,2,2,5,5,Yes,-,0,0,0,R,nan,SA,T Bosch,1992,1992


In [16]:
features = ["Hundreds","Fifties","Ducks","Runs","NO","Inns","HS"]
for name in features:
    for index, item in enumerate(df[name]):
        if str(item)=="-":
            df[name][index] = "0"

df["Hundreds"] = df["Hundreds"].astype("int8")
df["Fifties"] = df["Fifties"].astype("int8")
df["Ducks"] = df["Ducks"].astype("int8")
df["Runs"] = df["Runs"].astype("int16")
df["NO"] = df["NO"].astype("int8")
df["Inns"] = df["Inns"].astype("int16")
df["HS"] = df["HS"].astype("int16")
df["Mat"] = df["Mat"].astype("int16")

<ipython-input-16-4e00690275dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name][index] = "0"


In [17]:
df3 = df[df["Ave"]=="-"]
df3

,Mat,Inns,NO,Runs,HS,HS-N/O?,Ave,Hundreds,Fifties,Ducks,Batting Style,Player Role,Nationality,Player,StartDate,EndDate
1028,1,1,1,54,54,Yes,-,0,1,0,R,top-order batsman,AUS,SG Law,1995,1995
1236,1,2,2,16,12,Yes,-,0,0,0,R,bowler,PAK,Muhammad Musa,2019,2019
1262,3,4,4,13,8,Yes,-,0,0,0,R,nan,INDIA,T Yohannan,2001,2002
1296,1,1,1,9,9,Yes,-,0,0,0,R,nan,PAK,Iftikhar Anjum,2006,2006
1303,2,2,2,9,9,Yes,-,0,0,0,R,nan,PAK,Zakir Khan,1986,1989
1305,1,1,1,8,8,Yes,-,0,0,0,L,nan,AUS,PA Emery,1994,1994
1322,1,1,1,7,7,Yes,-,0,0,0,R,bowler,WI,DM Washington,2005,2005
1333,2,2,2,6,6,Yes,-,0,0,0,R,nan,AUS,SA Muller,1999,1999
1338,2,2,2,5,4,Yes,-,0,0,0,R,nan,WI,IBA Allen,1991,1991
1340,1,2,2,5,5,Yes,-,0,0,0,R,nan,SA,T Bosch,1992,1992


In [18]:
# Notice that Ave="-" if Inns=NO
# if Inns=NO=0, set Ave = 0
# if Inns=NO>0, set Ave = Runs/Inns
for index, item in enumerate(df.Ave):
    if str(item) == "-":
        if df.Inns[index] == 0 and df.NO[index] == 0:
            df.Ave[index] = "0"
        elif df.Inns[index] == df.NO[index] and float(df.Inns[index])>0:
            df.Ave[index] = float(df.Runs[index])/float(df.Inns[index])
df.Ave = df.Ave.astype("float16")

<ipython-input-18-f7b6d67159f9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Ave[index] = float(df.Runs[index])/float(df.Inns[index])
<ipython-input-18-f7b6d67159f9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Ave[index] = "0"


In [20]:
df.dtypes

Mat                int16
Inns               int16
NO                  int8
Runs               int16
HS                 int16
HS-N/O?           object
Ave              float16
Hundreds            int8
Fifties             int8
Ducks               int8
Batting Style     object
Player Role       object
Nationality       object
Player            object
StartDate          int16
EndDate            int16
dtype: object

In [21]:
for col in df.columns:
    if pd.api.types.is_string_dtype(df[col]):
        df[col] = pd.Categorical(df[col])

In [22]:
df.dtypes

Mat                 int16
Inns                int16
NO                   int8
Runs                int16
HS                  int16
HS-N/O?          category
Ave               float16
Hundreds             int8
Fifties              int8
Ducks                int8
Batting Style    category
Player Role      category
Nationality      category
Player           category
StartDate           int16
EndDate             int16
dtype: object

In [24]:
df["HS-N/O?"] = pd.Categorical(df["HS-N/O?"],categories=["No","Yes"],ordered=True)

In [37]:
# Can we order the player roles?
averole = df[["Player Role","Ave"]].groupby("Player Role").mean()
averole["Std. Dev"] = df[["Player Role","Ave"]].groupby("Player Role").std()
averole = averole.sort_values(by="Ave",ascending=False)
averole

,Ave,Std. Dev
Player Role,,
top-order batsman,34.031250,13.209580
middle-order batsman,33.312500,19.507223
batting allrounder,31.890625,14.810161
batsman,31.609375,12.866581
opening batsman,30.671875,12.227131
wicketkeeper batsman,28.031250,12.045926
bowling allrounder,24.921875,12.867174
wicketkeeper,21.421875,3.137527
nan,17.218750,12.294571


In [39]:
# Consider the order: top order > middle order > batsman > opener
# > batAllRounder > wicketBatsman > bowling allrounder > wicketkeeper
# > nan > bowler
rolcat = ["bowler","nan","wicketkeeper","bowling allrounder","wicketkeeper batsman","batting allrounder","opening batsman","batsman","middle-order batsman","top-order batsman"]
df["Player Role"] = pd.Categorical(df["Player Role"],categories=rolcat,ordered=True)
df["Player Role"]

0        top-order batsman
1        top-order batsman
2       bowling allrounder
3        top-order batsman
4          opening batsman
               ...        
1422                   nan
1423                   nan
1424                   nan
1425                bowler
1426                   nan
Name: Player Role, Length: 1427, dtype: category
Categories (10, object): ['bowler' < 'nan' < 'wicketkeeper' < 'bowling allrounder' ... 'opening batsman' < 'batsman' < 'middle-order batsman' < 'top-order batsman']

In [40]:
# Preprocessing done